In [ ]:

from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects
import keras
import tensorflow as tf

co = {}
_add_supported_quantized_objects(co)
import os
import plotting

experiment_id = "exp7_no_depth_w_GAP_multiObj"
trial_id = 719
model_path = f"../neural_network/automl/saved_models/striped_models/{experiment_id}/{trial_id}.h5"

In [ ]:
model = load_model(model_path, custom_objects=co, compile=False)

x = new_input = keras.Input((256,256,3))

for new_layer in model.layers[1:]:
    x = new_layer(x) 
model = keras.Model(inputs=new_input, outputs=x)


model.layers[-1].activation = keras.activations.linear
model.compile(
    "adam",
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
model.summary()
# new_model = model

In [ ]:
import hls4ml

config = hls4ml.utils.config_from_keras_model(
    model,
    granularity="name",
    default_precision="fixed<8,1>",
    default_reuse_factor=4096,
    
)

config["Model"]["Strategy"] = "Resource"

for layer in config["LayerName"].keys():
    config["LayerName"][layer]["Strategy"] = "Resource"


In [ ]:
print("-----------------------------------")
print("Configuration")
plotting.print_dict(config)
print("-----------------------------------")

In [ ]:

hls_model = hls4ml.converters.convert_from_keras_model(
    model,
    hls_config=config,
    output_dir=f"deploy/model_{trial_id}/hls4ml_prj",
    part="xc7z020-clg400-1",
    io_type="io_stream",
    backend='VivadoAccelerator',
    board='pynq-z2'
)

In [ ]:
hls_model.build(csim=False, export=True, bitfile=True)

In [ ]:
hls4ml.report.read_vivado_report(f"deploy/model_{trial_id}/hls4ml_prj")